In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import cont2discrete, tf2ss
import math

zeta = 0.6
omega_n = 10

# Transfer function coefficients
num = [omega_n**2]
den = [1, 2*zeta*omega_n, omega_n**2]

# Convert TF → State-space
A, B, C, D = tf2ss(num, den)

Ts = 0.01  # sample time

# Now discretize the state-space system
Ad, Bd, Cd, Dd, dt = cont2discrete((A, B, C, D), Ts, method='bilinear')

print(A, B, C, D)
# Initial state
x = np.zeros((2, 1))

def pitch_update(beta_ref):
    global x

    x = Ad @ x + Bd * beta_ref

    beta = Cd @ x + Dd * beta_ref
    return float(beta)

beta_values = []
for k in range(100):
    beta = pitch_update(beta_ref=4.0)   # command pitch = 5 degrees
    beta_values.append(beta)

#plt.plot(beta_values)

[[ -12. -100.]
 [   1.    0.]] [[1.]
 [0.]] [[  0. 100.]] [[0.]]


C:\Users\claud\AppData\Local\Temp\ipykernel_15796\1100560800.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(beta)


In [26]:
path = r'TSRxCPxPSI.csv'
df = pd.read_csv(path)
df

,TSR,-5,-4,-3,-2,-1,0,1,2,3,4,5
0,0.411942,0.0007,0.0010,0.0013,0.0015,0.0018,0.0021,0.0023,0.0026,0.0028,0.0031,0.0033
1,0.439404,0.0008,0.0011,0.0014,0.0016,0.0019,0.0022,0.0025,0.0027,0.0030,0.0033,0.0035
2,0.470790,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.0030,0.0032,0.0035,0.0038
3,0.507005,0.0009,0.0013,0.0016,0.0019,0.0022,0.0026,0.0029,0.0032,0.0035,0.0038,0.0041
4,0.549255,0.0010,0.0014,0.0017,0.0021,0.0024,0.0028,0.0032,0.0035,0.0038,0.0042,0.0045
...,...,...,...,...,...,...,...,...,...,...,...,...
191,23.068729,-0.5313,-0.5180,-0.5276,-0.5396,-0.5539,-0.5790,-0.6162,-0.6788,-0.8500,-1.1788,-1.7242
192,24.167240,-0.6381,-0.6257,-0.6392,-0.6585,-0.6841,-0.7269,-0.7918,-0.8824,-1.0819,-1.4697,-2.1104
193,26.364261,-0.8841,-0.8743,-0.8973,-0.9319,-0.9832,-1.0642,-1.1895,-1.3443,-1.6264,-2.1537,-3.0514
194,28.561283,-1.1772,-1.1713,-1.2051,-1.2564,-1.3369,-1.4665,-1.6582,-1.9132,-2.2992,-3.0409,-4.1698


In [27]:
path = r'TSRxCPxPSI.csv'
df = pd.read_csv(path)
angles = np.arange(-5,6,1)
TSR = df.iloc[:,0]

def Cp_calc(LAMBDA=0, PHI=0):
    path = r'TSRxCPxPSI.csv'
    df = pd.read_csv(path)
    angles = np.arange(-5,6,1)
    TSR = df.iloc[:,0]
    diff1 = np.abs(TSR - LAMBDA)
    diff2 = np.abs(angles - PHI)
    id1 = np.argmin(diff1)
    id2 = np.argmin(diff2)+1

    return df.iloc[id1,id2]

Cp_calc(LAMBDA=7.6, PHI=1)

0.4796

In [72]:
import scipy.io
mat_file_path = 'Cq.mat'

# Load the .mat file
mat_data = scipy.io.loadmat(mat_file_path)
#print(mat_data)
Cq = mat_data['Cq'].T
tsr = np.array([1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10,10.5,11,11.5,12,12.5,13])
Cq =np.vstack([tsr,Cq]).T
df = pd.DataFrame(Cq)
df.columns = np.array(['TSR',-2,-1.5,-1,-0.75,-0.5,-0.25,0,0.25,0.5,0.75,1,1.5,2,3,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95])
df.to_csv('Cq.csv',index=False)


In [87]:
import plotly.graph_objects as go
path = r'Cq.csv'
df = pd.read_csv(path)
X = df.iloc[:, 0].values
Y = np.arange(len(df.columns)-1)
X, Y = np.meshgrid(X, Y)

Z = np.array(df.iloc[:, 1:]).T

# Create the plot
fig = go.Figure(data=[go.Surface(z=Z, x=Y, y=X, colorscale='magma')])

# Customize layout
fig.update_layout(
    scene = {
        'xaxis_title': 'Pitch angle',
        'yaxis_title': 'TSR',
        'zaxis_title': 'Z Axis',
        #'zaxis': dict(range=[-2.01, 0.5])
    },
    coloraxis_colorbar=dict(
        title="Color Scale"
    ),
)

# Show the plot
fig.show()

In [ ]:

params = {
    
    'ALPHA' : 0.1,
    'ALPHA_gc' : 50,
    'Area' :  math.pi * (57.5**2),
    'B_dt' : 755.49,
    'B_r' : 7.11,
    'B_g' : 45.6,
    'CP_max' : 0.486,
    'D_max' : 1000,
    'ETA_dt' : 0.97,
    'ETA_gc' : 0.98,
    'f_s': 0.0001,
    'H' : 81,
    'J_r' : 55*1e6,
    'J_g' : 390,
    'K_dt' : 2.7*1e9,
    'K_opt' : 1.2171,
    'k_i' : 1,
    'k_p' : 4,
    'LAMBDA_opt' : 7.6,
    'N_g' : 95,
    'OMEGA_n' : 11.11,
    'OMEGA_nom' : 162,
    'OMEGA_var' : 15,
    'PI' : np.pi,
    'PHI_opt' : 0,
    'P_r' : 4.8*1e6,
    'r_0' : 1.5,
    'R' : 57.5,
    'RHO' : 1.225,
    'T_s' : 0.01,
    'ZETA' : 0.6,

    #'' : ,

}

df = pd.DataFrame(list(params.items()), columns=['Parameter', 'Value'])

# Save DataFrame to CSV
df.to_csv('params2.csv', index=False)

In [3]:
df_loaded = pd.read_csv('params.csv')

# Convert DataFrame back to dictionary
params_loaded = dict(zip(df_loaded['Parameter'], df_loaded['Value']))

# Show the loaded dictionary
print(params_loaded)

{'ALPHA': 0.1, 'ALPHA_gc': 50.0, 'Area': 10386.890710931251, 'B_dt': 755.49, 'B_r': 0.0, 'B_g': 0.0, 'CP_max': 0.486, 'D_max': 1000.0, 'ETA_dt': 1.0, 'ETA_gc': 1.0, 'f_s': 0.01, 'H': 81.0, 'J_r': 55000000.0, 'J_g': 55000000.0, 'K_dt': 2700000000.0, 'K_opt': 1.2171, 'k_i': 1.0, 'k_p': 4.0, 'LAMBDA_opt': 7.6, 'N_g': 1.0, 'OMEGA_n': 11.11, 'OMEGA_nom': 5.0, 'OMEGA_var': 2.8, 'PI': 3.141592653589793, 'PHI_opt': 0.0, 'P_r': 22500000.0, 'r_0': 1.5, 'R': 57.5, 'RHO': 1.225, 'T_s': 1.0, 'ZETA': 0.6}


In [19]:
path = r'Cp.csv'
df = pd.read_csv(path)
print(df.iloc[13:16,0:9])
df.iloc[:,3:10].max()

    TSR      -2    -1.5      -1   -0.75    -0.5   -0.25       0    0.25
13  7.5  0.4509  0.4527  0.4536  0.4536  0.4527  0.4518  0.4500  0.4482
14  8.0  0.4410  0.4491  0.4536  0.4554  0.4554  0.4554  0.4554  0.4536
15  8.5  0.4167  0.4347  0.4455  0.4491  0.4518  0.4536  0.4545  0.4545


-1       0.4536
-0.75    0.4554
-0.5     0.4554
-0.25    0.4554
0        0.4554
0.25     0.4545
0.5      0.4536
dtype: float64

In [23]:
path = r'Cq.csv'
df = pd.read_csv(path)
print(df.iloc[10:13,0:9])
df.iloc[:,3:9].max()

    TSR      -2    -1.5      -1   -0.75    -0.5   -0.25       0    0.25
10  6.0  0.0594  0.0612  0.0621  0.0630  0.0630  0.0630  0.0630  0.0630
11  6.5  0.0630  0.0639  0.0648  0.0648  0.0648  0.0648  0.0648  0.0639
12  7.0  0.0639  0.0639  0.0630  0.0630  0.0630  0.0630  0.0630  0.0621


-1       0.0648
-0.75    0.0648
-0.5     0.0648
-0.25    0.0648
0        0.0648
0.25     0.0639
dtype: float64

In [31]:
path = r'TSRxCPxPSI.csv'
df = pd.read_csv(path)
print(df.iloc[139:142,0:8])
df.iloc[:,5:8].max()

          TSR      -5      -4      -3      -2      -1       0       1
139  7.789441  0.4041  0.4253  0.4468  0.4668  0.4811  0.4865  0.4810
140  7.909278  0.3992  0.4218  0.4444  0.4647  0.4792  0.4859  0.4814
141  7.909278  0.3992  0.4218  0.4444  0.4647  0.4792  0.4859  0.4814


-1    0.4844
0     0.4868
1     0.4815
dtype: float64

In [39]:
def Cp_calc(LAMBDA=0, PHI=0):
    PHI = PHI*180/np.pi
    path = r'Cp.csv'
    df = pd.read_csv(path)
    angles = np.arange(-5,6,1)
    TSR = df.iloc[:,0]
    diff1 = np.abs(TSR - LAMBDA)
    diff2 = np.abs(angles - PHI)
    id1 = np.argmin(diff1)
    id2 = np.argmin(diff2)+1
    return df.iloc[id1,id2]

In [41]:
Cp_calc(LAMBDA=7.6, PHI=0)


0.4518

In [38]:
Cp_calc(LAMBDA=7.9, PHI=0)


0.4859